In [1]:
import pandas as pd
import numpy as np

from pymongo import MongoClient

import re
#fixed issue with ascii encoding errors
import sys

# reload(sys)
# sys.setdefaultencoding("utf-8")
import ExtractHelper as eh

In [2]:
#code to put data from mongodb named Poetry into two separate Pandas DataFrame
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client.POETRY

poets = db.POET

rawpoet = pd.DataFrame(list(poets.find()))

In [3]:
rawpoet.to_pickle('/Users/huiru/GADataScience/FinalProject/Data/rawpoet.pkl')

### clean rawpoet table. At the moment, _id, poet_positions, & posyears (position years) column values appear unusable
### drop these columns for now

In [4]:
ctodrop = ['_id', 'poet_positions', 'poet_posyears']
rawpoet.drop(ctodrop, 1,inplace = True)

In [5]:
#These are the columns that contain list of strings or strings that 
#need to be turned into one string for cleaning/extraction

ctostr = ['movement_name','poet_bio','poet_name', 'poet_basicbio', 'poet_tags']

for i in ctostr:
    rawpoet[i] = rawpoet[i].map(lambda x: u''.join(x))

In [10]:
# rawpoet.head()[:1]

###the poet's movement name tag is very important, we need to clean (remove dup tags), separate out into columns, then merge back into a clean poet table

In [11]:
rawpoet['tags'] = rawpoet[['movement_name', 'poet_tags']].apply(lambda x: list(set(str(x))),axis=1)
rawpoet['tags'] = rawpoet[['movement_name', 'poet_tags']].apply(lambda x: list(set(x)),axis=1)
rawpoet['tags'] = rawpoet['tags'].apply(lambda x: x.replace('-','').replace(' ','').split("\n"))
rawpoet['tags']=rawpoet['tags'].map(lambda x: ''.join(x).split(","))

In [12]:
# rawpoet.tags[:10]

In [13]:
#Split out the list of tags from the tags column and put into a data frame to then concat 
#back into a final cleaned poet dataframe
ptags = rawpoet['tags'].apply(pd.Series)
ptags.columns = ['primary_tag', 'tag2', 'tag3','tag4']
# ptags.head()

##now we need to look at poet births/deaths

In [14]:
?eh.extract_birth()

In [15]:
#use the helper function extract_birth to fill the poet birth yr, yr died (if dead) variables
rawpoet['yr_born'] = rawpoet['poet_dob2'].apply(lambda x: eh.extract_birth(x))
rawpoet['yr_died'] = rawpoet['poet_dob2'].apply(lambda x: eh.extract_birth(x,False))

In [16]:
#put the poet_basicbio, which is a string, into another cleaned string column to work with
rawpoet['basic_bio']=rawpoet['poet_basicbio'].apply(lambda x: x.replace('-',''))

poet_basicbio contains a secondary source of birth and death years. It also contains where the poet was born; city, state and country. We need to extract this information.

In [17]:
#extract anything that is a number
rawpoet['birthdeath'] = rawpoet['basic_bio'].map(lambda x: re.findall(r'\d+', x))

#extract non-numeric characters
rawpoet['birthplace'] = rawpoet['basic_bio'].map(lambda x: re.findall(r'[^\d]+', x))
rawpoet['birthplace'] = rawpoet['birthplace'].apply(lambda x: u''.join(x))

In [18]:
?eh.extract_bio()

In [19]:
rawpoet['city'] = rawpoet['birthplace'].map(lambda x: eh.extract_bio(x))
rawpoet['state'] = rawpoet['birthplace'].map(lambda x: eh.extract_bio(x, 'state'))
rawpoet['country'] = rawpoet['birthplace'].map(lambda x: eh.extract_bio(x, 'country'))

In [20]:
#Now the data is more or less ready to put in cleaned dataframe of poet information
#rawpoet.columns

coltokeep= ['poet_name','poet_url','poet_poems_url','tags','poet_bio','yr_born','yr_died','city','state','country']

poetdf = pd.concat([rawpoet[coltokeep],ptags],axis=1)

In [21]:
#Not sure if I want to keep these variables calculated from the birth and death information about each poet
#using them for now
#It's 2015 now, so use this to calculate current age if poet hasn't died

poetdf['dead'] = np.where(poetdf['yr_died'] > 0,1,0)
poetdf['current_age'] = np.where(poetdf['dead'] == 0, (2015- poetdf['yr_born']), 0)
poetdf['age_dead'] = np.where(poetdf['dead'] == 1, (poetdf['yr_died']-poetdf['yr_born']), 0)
poetdf['current_age'] = np.where(poetdf['current_age'] == 2015, 0,poetdf['current_age'])

In [22]:
poetdf.head()[:1]

#Combine cleaned poet dataframe and poem data frame based on the foreign key identifier, poet_poems_url
# rawpoem.head()
# poemdf = rawpoem[['poet_poems_url','poem_title', 'poem_yrpub', 'poem_text', 'poem_link']]
# resultdf = pd.merge(poetdf, poemdf, how='inner', on=['poet_poems_url'])

,poet_name,poet_url,poet_poems_url,tags,poet_bio,yr_born,yr_died,city,state,country,primary_tag,tag2,tag3,tag4,dead,current_age,age_dead
0,Taylor Mali,http://www.poets.org/poetsorg/poet/taylor-mali,http://www.poets.org/poetsorg/poems/406436,[Slam/SpokenWord],Taylor Mali is a four-time National Poetry Sla...,0,0,,,,Slam/SpokenWord,NaN,NaN,NaN,0,0,0


In [23]:
#Save the results to a pickle file. Alternatively, can also save to a collection in the same POETRY database
poetdf.to_pickle('/Users/huiru/GADataScience/FinalProject/Data/poetdf.pkl')